# Optimization Project 1
### Done by Charan Musunuru, Sahil Natu, Jacob Rhymes, Aditya Soni

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp

In [2]:
#reading the csv files
roi_data=pd.read_csv('ROI_data.csv')
roi_data.set_index('Platform', inplace = True)
roi_mat=pd.read_csv('roi_mat.csv', index_col=0)/100


In [3]:
display(roi_data)
display(roi_mat)

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Platform,,,,,,,,,,
ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
January,0.040,0.036,0.024,0.039,0.030,0.035,0.036,0.0225,0.035,0.035
February,0.040,0.039,0.027,0.038,0.043,0.032,0.027,0.0180,0.037,0.035
March,0.035,0.029,0.031,0.038,0.024,0.041,0.037,0.0260,0.042,0.025
April,0.038,0.031,0.024,0.044,0.024,0.038,0.037,0.0250,0.036,0.029
May,0.035,0.032,0.019,0.034,0.027,0.027,0.039,0.0220,0.045,0.039
June,0.040,0.032,0.027,0.034,0.034,0.030,0.045,0.0210,0.038,0.041
July,0.039,0.036,0.020,0.044,0.039,0.037,0.043,0.0180,0.040,0.038
August,0.042,0.033,0.028,0.042,0.020,0.037,0.036,0.0150,0.044,0.043
September,0.041,0.028,0.025,0.042,0.029,0.037,0.028,0.0250,0.040,0.034
October,0.030,0.030,0.031,0.046,0.031,0.033,0.032,0.0230,0.025,0.032


### 3rd Question

In [4]:
#creating the objective function for the first consulting firm
obj = np.array(roi_data.loc['ROI'].tolist())

In [5]:
#creating the constraint matrix
A = np.zeros((13,10))
A[0,:] = [1,1,1,1,1,1,1,1,1,1]      #Total sum of all money spent be less that or equal to 10 million USD (budget)
A[1,:] = [1,1,0,0,-1,0,0,0,0,-1]    #Amount of money spent on print and TV be less than money spent on facebook and email
A[2,:] = [0,0,-2,-2,1,1,1,1,1,0]    #Total amount used in Social media atleast twice of SEO and adword 
A[3:,:] = np.diag(np.ones(10))      #Money spent on any medium be less than or equal to 3 million US
b = np.array([10000000,0,0,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000])
sense = np.array(['<','<','>','<','<','<','<','<','<','<','<','<','<'])

In [6]:
#running gurobi
ojModel = gp.Model() 

ojModX = ojModel.addMVar(10) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

Academic license - for non-commercial use only - expires 2022-08-27
Using license file C:\Users\sahil\gurobi.lic


In [7]:
df = pd.DataFrame(columns = roi_data.columns.values.tolist() + ['Optimal ROI'])
df.loc['First Consulting Firm'] = ojModel.x + [ojModel.objval]

In [8]:
print("The allocation for each medium and the optimal ROI is")
df

The allocation for each medium and the optimal ROI is


,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Optimal ROI
First Consulting Firm,0.0,3000000.0,0.0,1000000.0,0.0,0.0,3000000.0,0.0,0.0,3000000.0,456000.0


### 4th Question

In [9]:
#The objective function for the second consulting firm
obj = np.array(roi_data.loc['Second Firms ROI Estimate'].tolist())

In [10]:
#creating the constraint matrix
A = np.zeros((13,10))
A[0,:] = [1,1,1,1,1,1,1,1,1,1]      #Total sum of all money spent be less that or equal to 10 million USD (budget)
A[1,:] = [1,1,0,0,-1,0,0,0,0,-1]    #Amount of money spent on print and TV be less than money spent on facebook and email
A[2,:] = [0,0,-2,-2,1,1,1,1,1,0]    #Total amount used in Social media atleast twice of SEO and adword 
A[3:,:] = np.diag(np.ones(10))      #Money spent on any medium be less than or equal to 3 million US
b = np.array([10000000,0,0,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000])
sense = np.array(['<','<','>','<','<','<','<','<','<','<','<','<','<'])

In [11]:
#running gurobi
ojModel = gp.Model() 

ojModX = ojModel.addMVar(10) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [12]:
df.loc['Second Consulting Firm'] = ojModel.x + [ojModel.objval]

In [13]:
print("The optimal allocations for the First Consulting firm and the second consulting firms \
are given below. The optimal returns remain the same")
df

The optimal allocations for the First Consulting firm and the second consulting firms are given below. The optimal returns remain the same


,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Optimal ROI
First Consulting Firm,0.0,3000000.0,0.0,1000000.0,0.0,0.0,3000000.0,0.0,0.0,3000000.0,456000.0
Second Consulting Firm,3000000.0,0.0,0.0,1000000.0,3000000.0,3000000.0,0.0,0.0,0.0,0.0,456000.0


### 5th Question

The allocations are different though the Optimal returns are the same

In [14]:
difference_1 = df.loc['First Consulting Firm','Optimal ROI'] - np.array(roi_data.loc['ROI'].tolist()) @ np.array(df.iloc[1,:-1].tolist())
print("Assuming the first ROI data is correct, if we were to use the second allocation (the allocation that assumed the second ROI data was correct)\
the objective relative to the optimal objective would lower by: " + str(difference_1))
print("=====================================================================================================================")
difference_2 = df.loc['Second Consulting Firm','Optimal ROI'] -  np.array(roi_data.loc['Second Firms ROI Estimate'].tolist()) @ np.array(df.iloc[0,:-1].tolist())
print("Assuming the second ROI data is correct, if we were to use the first allocation (the allocation that assumed the second ROI data was correct)\
the objective relative to the optimal objective would lower by: " + str(difference_2))

Assuming the first ROI data is correct, if we were to use the second allocation (the allocation that assumed the second ROI data was correct)the objective relative to the optimal objective would lower by: 204000.0
Assuming the second ROI data is correct, if we were to use the first allocation (the allocation that assumed the second ROI data was correct)the objective relative to the optimal objective would lower by: 192000.0


Is the third constraint useful?

In [15]:
#First Consulting firm
obj = np.array(roi_data.loc['ROI'].tolist())

A = np.zeros((3,10))
A[0,:] = [1,1,1,1,1,1,1,1,1,1]      #Total sum of all money spent be less that or equal to 10 million USD (budget)
A[1,:] = [1,1,0,0,-1,0,0,0,0,-1]    #Amount of money spent on print and TV be less than money spent on facebook and email
A[2,:] = [0,0,-2,-2,1,1,1,1,1,0]    #Total amount used in Social media atleast twice of SEO and adword 
b = np.array([10000000,0,0])
sense = np.array(['<','<','>'])

ojModel = gp.Model() 

ojModX = ojModel.addMVar(10) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()


In [16]:
temp_df = pd.DataFrame(columns = roi_data.columns.values.tolist() + ['Optimal ROI'])
temp_df.loc['First Consulting firm'] = ojModel.x + [ojModel.objval]

In [17]:
#Second Consulting firm
obj = np.array(roi_data.loc['Second Firms ROI Estimate'].tolist())

A = np.zeros((3,10))
A[0,:] = [1,1,1,1,1,1,1,1,1,1]      #Total sum of all money spent be less that or equal to 10 million USD (budget)
A[1,:] = [1,1,0,0,-1,0,0,0,0,-1]    #Amount of money spent on print and TV be less than money spent on facebook and email
A[2,:] = [0,0,-2,-2,1,1,1,1,1,0]    #Total amount used in Social media atleast twice of SEO and adword 
b = np.array([10000000,0,0])
sense = np.array(['<','<','>'])

ojModel = gp.Model() 

ojModX = ojModel.addMVar(10) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [18]:
print("The new optimal ROI is " + str(ojModel.objval))
print("The allocation is")
temp_df.loc['Second Consulting firm'] = ojModel.x + [ojModel.objval]
temp_df

The new optimal ROI is 465000.0
The allocation is


,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Optimal ROI
First Consulting firm,0.0,5000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,465000.0
Second Consulting firm,5000000.0,0.0,0.0,0.0,5000000.0,0.0,0.0,0.0,0.0,0.0,465000.0


The new optimal ROI is 46500 which is 9000 greater than with the constraint

In [22]:
difference_1 = df.loc['First Consulting Firm','Optimal ROI'] - np.array(roi_data.loc['ROI'].tolist()) @ np.array(temp_df.iloc[1,:-1].tolist())
print("Assuming the first ROI data is correct, if we were to use the second allocation strategy (without 3rd constraint) \
the objective relative to the optimal objective would lower by: " + str(difference_1))
difference_2 = df.loc['Second Consulting Firm','Optimal ROI'] -  np.array(roi_data.loc['Second Firms ROI Estimate'].tolist()) @ np.array(temp_df.iloc[0,:-1].tolist())
print("Assuming the second ROI data is correct, if we were to use the first allocation strategy (without 3rd constraint) \
the objective relative to the optimal objective would lower by: " + str(difference_2))
print("While the optimal ROI without 3rd constraint increased by 9000, the reduction in optimal ROI was much higher if the estimate \
was wrong. The 3rd constraint thus minimizes the risk even though the optimal ROI is reduced a little.")

Assuming the first ROI data is correct, if we were to use the second allocation strategy (without 3rd constraint) the objective relative to the optimal objective would lower by: 221000.0
Assuming the second ROI data is correct, if we were to use the first allocation strategy (without 3rd constraint) the objective relative to the optimal objective would lower by: 211000.0
While the optimal ROI without 3rd constraint increased by 9000, the reduction in optimal ROI was much higher if the estimate was wrong. The 3rd constraint thus minimizes the risk even though the optimal ROI is reduced a little.


### 6th Question

In [23]:
#objective function
obj = np.array(roi_data.loc['ROI'].tolist())

#constraint matrix
A = np.zeros((13,10))
A[0,:] = [1,1,1,1,1,1,1,1,1,1]      #Total sum of all money spent be less that or equal to 10 million USD (budget)
A[1,:] = [1,1,0,0,-1,0,0,0,0,-1]    #Amount of money spent on print and TV be less than money spent on facebook and email
A[2,:] = [0,0,-2,-2,1,1,1,1,1,0]    #Total amount used in Social media atleast twice of SEO and adword 
A[3:,:] = np.diag(np.ones(10))      #Money spent on any medium be less than or equal to 3 million US
b = np.array([10000000,0,0,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000])
sense = np.array(['<','<','>','<','<','<','<','<','<','<','<','<','<'])

In [24]:
#running gurobi
ojModel = gp.Model() 

ojModX = ojModel.addMVar(10) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

ojModel.Params.OutputFlag = 0 
ojModel.optimize()

In [25]:
temp_df = pd.DataFrame(columns = roi_data.columns.values.tolist())
temp_df.loc['Max Values'] = ojModX.SAObjUp
temp_df.loc['Min Values'] = ojModX.SAObjLow


In [26]:
temp_df

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Max Values,0.049,0.062,0.039,0.046,0.029,0.039,inf,0.039,0.039,inf
Min Values,-inf,0.039,-inf,0.033,-inf,-inf,0.039,-inf,-inf,0.029


### 7th question

In [27]:
obj = np.array(roi_data.loc['ROI'].tolist())
temp_df = pd.DataFrame(columns = roi_data.columns.values.tolist() + ['Optimal ROI'] + ['Budget'])
A = np.zeros((13,10))
A[0,:] = [1,1,1,1,1,1,1,1,1,1]      #Total sum of all money spent be less that or equal to 10 million USD (budget)
A[1,:] = [1,1,0,0,-1,0,0,0,0,-1]    #Amount of money spent on print and TV be less than money spent on facebook and email
A[2,:] = [0,0,-2,-2,1,1,1,1,1,0]    #Total amount used in Social media atleast twice of SEO and adword 
A[3:,:] = np.diag(np.ones(10))  
constraint1 = 10000000
for i in range(12):
    b = np.array([constraint1, 0,0,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000,3000000])
    sense = np.array(['<','<','>','<','<','<','<','<','<','<','<','<','<'])
    ojModel = gp.Model() 

    ojModX = ojModel.addMVar(10) 
    ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
    ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 

    ojModel.Params.OutputFlag = 0 
    ojModel.optimize()
    roi = np.array(ojModel.x)  @ np.array(roi_mat.iloc[i])
    temp_df.loc[i] = ojModel.x + [roi] + [constraint1]
    #print(constraint1)
    constraint1 = roi/2 + constraint1 
   
    

temp_df.index = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec']

In [28]:
temp_df

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Optimal ROI,Budget
Jan,0.0,3000000.0,0.0,1.000000e+06,0.0,0.0,3000000.0,0.0,0.000000e+00,3000000.0,360000.000000,1.000000e+07
Feb,0.0,3000000.0,0.0,1.180000e+06,0.0,0.0,3000000.0,0.0,0.000000e+00,3000000.0,347840.000000,1.018000e+07
Mar,0.0,3000000.0,0.0,1.353920e+06,0.0,0.0,3000000.0,0.0,0.000000e+00,3000000.0,324448.960000,1.035392e+07
Apr,0.0,3000000.0,0.0,1.505381e+06,0.0,0.0,3000000.0,0.0,1.076299e+04,3000000.0,357624.253227,1.051614e+07
May,0.0,3000000.0,0.0,1.564986e+06,0.0,0.0,3000000.0,0.0,1.299711e+05,3000000.0,389058.206407,1.069496e+07
June,0.0,3000000.0,0.0,1.629829e+06,0.0,0.0,3000000.0,0.0,2.596571e+05,3000000.0,419281.142693,1.088949e+07
July,0.0,3000000.0,0.0,1.699709e+06,0.0,0.0,3000000.0,0.0,3.994175e+05,3000000.0,441763.886288,1.109913e+07
Aug,0.0,3000000.0,0.0,1.773336e+06,0.0,0.0,3000000.0,0.0,5.466721e+05,3000000.0,434533.689720,1.132001e+07
Sep,0.0,3000000.0,0.0,1.845758e+06,0.0,0.0,3000000.0,0.0,6.915167e+05,3000000.0,375182.519479,1.153728e+07
Oct,0.0,3000000.0,0.0,1.908289e+06,0.0,0.0,3000000.0,0.0,8.165776e+05,3000000.0,390195.722525,1.172487e+07


### 8th Question

It is a stable budget